In [1]:
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

>
> These queries come from Wikidata's [SPARQL query examples](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)
>

# Simple queries

## Cats, with pictures

In [2]:
%%wdsparql
#defaultView:ImageGrid
SELECT ?item ?itemLabel ?pic
WHERE
{
?item wdt:P31 wd:Q146 .
?item wdt:P18 ?pic
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }
}

,item,itemLabel,pic
0,http://www.wikidata.org/entity/Q49581026,Toffee,http://commons.wikimedia.org/wiki/Special:File...
1,http://www.wikidata.org/entity/Q51596094,Crimean Tom,http://commons.wikimedia.org/wiki/Special:File...
2,http://www.wikidata.org/entity/Q53216650,Nitama,http://commons.wikimedia.org/wiki/Special:File...
3,http://www.wikidata.org/entity/Q54087698,Foss,http://commons.wikimedia.org/wiki/Special:File...
4,http://www.wikidata.org/entity/Q61133276,Şero,http://commons.wikimedia.org/wiki/Special:File...
5,http://www.wikidata.org/entity/Q62128317,Naro,http://commons.wikimedia.org/wiki/Special:File...
6,http://www.wikidata.org/entity/Q66737405,Mittens,http://commons.wikimedia.org/wiki/Special:File...
7,http://www.wikidata.org/entity/Q76974430,Apelmon,http://commons.wikimedia.org/wiki/Special:File...
8,http://www.wikidata.org/entity/Q94990539,Pip,http://commons.wikimedia.org/wiki/Special:File...
9,http://www.wikidata.org/entity/Q105330885,Tuna,http://commons.wikimedia.org/wiki/Special:File...


In [3]:
_dfwd

,item,itemLabel,pic
0,http://www.wikidata.org/entity/Q49581026,Toffee,http://commons.wikimedia.org/wiki/Special:File...
1,http://www.wikidata.org/entity/Q51596094,Crimean Tom,http://commons.wikimedia.org/wiki/Special:File...
2,http://www.wikidata.org/entity/Q53216650,Nitama,http://commons.wikimedia.org/wiki/Special:File...
3,http://www.wikidata.org/entity/Q54087698,Foss,http://commons.wikimedia.org/wiki/Special:File...
4,http://www.wikidata.org/entity/Q61133276,Şero,http://commons.wikimedia.org/wiki/Special:File...
5,http://www.wikidata.org/entity/Q62128317,Naro,http://commons.wikimedia.org/wiki/Special:File...
6,http://www.wikidata.org/entity/Q66737405,Mittens,http://commons.wikimedia.org/wiki/Special:File...
7,http://www.wikidata.org/entity/Q76974430,Apelmon,http://commons.wikimedia.org/wiki/Special:File...
8,http://www.wikidata.org/entity/Q94990539,Pip,http://commons.wikimedia.org/wiki/Special:File...
9,http://www.wikidata.org/entity/Q105330885,Tuna,http://commons.wikimedia.org/wiki/Special:File...


## Number of humans in Wikidata

In [4]:
%%wdsparql
#title: Number of humans in Wikidata
SELECT (COUNT(*) AS ?count)
WHERE {
  ?item wdt:P31 wd:Q5 .
}

,count
0,9712292


## Recent events

In [5]:
%%wdsparql
#title: Recent events
SELECT ?event ?eventLabel ?date
WHERE
{
  # find events
  ?event wdt:P31/wdt:P279* wd:Q1190554.
  # with a point in time or start date
  OPTIONAL { ?event wdt:P585 ?date. }
  OPTIONAL { ?event wdt:P580 ?date. }
  # but at least one of those
  FILTER(BOUND(?date) && DATATYPE(?date) = xsd:dateTime).
  # not in the future, and not more than 31 days ago
  BIND(NOW() - ?date AS ?distance).
  FILTER(0 <= ?distance && ?distance < 31).
  # and get a label as well
  OPTIONAL {
    ?event rdfs:label ?eventLabel.
    FILTER(LANG(?eventLabel) = "en").
  }
}
# limit to 5 results so we don't timeout
LIMIT 5

,event,eventLabel,date
0,http://www.wikidata.org/entity/Q29468697,r/place,2022-04-01T00:00:00Z
1,http://www.wikidata.org/entity/Q29468697,r/place,2022-04-01T00:00:00Z
2,http://www.wikidata.org/entity/Q51224536,date of Easter,2022-04-17T00:00:00Z
3,http://www.wikidata.org/entity/Q104319778,2020 Invictus Games,2022-04-01T00:00:00Z
4,http://www.wikidata.org/entity/Q104924327,2022 Serbian general election,2022-04-03T00:00:00Z


## Popular eye colors among humans

In [6]:
%%wdsparql
#title:Popular eye colors among humans
#illustrates bubblechart view, count

#defaultView:BubbleChart
SELECT ?eyeColorLabel (COUNT(?human) AS ?count)
WHERE
{
  ?human wdt:P31 wd:Q5.
  ?human wdt:P1340 ?eyeColor.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY ?eyeColorLabel

,eyeColorLabel,count
0,blue-green,99
1,metal,1
2,sky blue,3
3,coffee,9
4,gold,1
5,horizon blue,1
6,Marron,1
7,olive,1
8,Negros,1
9,honey,1


## Mayors that are any kind of domesticated animal

In [7]:
%%wdsparql
#title: Mayors that are any kind of domesticated animal
SELECT ?image ?speciesLabel ?mayorLabel ?placeLabel WHERE {
  ?species wdt:P279* wd:Q622852 .
  ?mayor wdt:P31 ?species . 
 
  ?mayor p:P39 ?node .
  ?node ps:P39 wd:Q30185 .
  ?node pq:P642 ?place .
  optional{?mayor wdt:P18 ?image}
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}

,image,speciesLabel,mayorLabel,placeLabel
0,http://commons.wikimedia.org/wiki/Special:File...,dog,Bosco the dog,Sunol
1,None,dog,Duke the Dog,Cormorant Township
